In [ ]:
import model as m

In [1]:
# import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, BatchNormalization, Activation, Add, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg19 import VGG19


# Define the generator network
def build_generator():
    # Input layer
    input_layer = Input(shape=(None, None, 3))
    
    # First convolutional block
    x = Conv2D(64, kernel_size=9, padding='same')(input_layer)
    x = Activation('relu')(x)
    
    # Residual blocks
    for i in range(16):
        residual = x
        x = Conv2D(64, kernel_size=3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(64, kernel_size=3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Add()([x, residual])
    
    # Second convolutional block
    x = Conv2D(64, kernel_size=3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, input_layer])
    
    # Upsampling blocks
    x = UpSampling2D(size=2)(x)
    x = Conv2D(256, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = UpSampling2D(size=2)(x)
    x = Conv2D(256, kernel_size=3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(3, kernel_size=9, padding='same')(x)
    output_layer = Activation('tanh')(x)
    
    # Define the model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

# Define the discriminator network
def build_discriminator():
    # Input layer
    input_layer = Input(shape=(None, None, 3))
    
    # Convolutional blocks
    x = Conv2D(64, kernel_size=3, strides=1, padding='same')(input_layer)
    x = Activation('relu')(x)
    x = Conv2D(64, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, kernel_size=3, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Dense layer
    x = Dense(1024)(x)
    x = Activation('relu')(x)
    output_layer = Dense(1, activation='sigmoid')(x)
    
    # Define the model
    model = Model(inputs=input_layer, outputs=output_layer)
    
    return model

# Define the SRGAN model
def build_srgan(generator, discriminator):
    # Freeze the discriminator weights
    discriminator.trainable = False
    
    # Input layer
    input_layer = Input(shape=(None, None, 3))
    
    # Generate high-resolution image
    generated_image = generator(input_layer)
    
    # Extract features from generated image
    features = VGG19(include_top=False, weights='imagenet')(generated_image)
    
    # Define the combined model
    model = Model(inputs=input_layer, outputs=[generated_image, discriminator(generated_image), features])
    
    # Compile the model
    optimizer = Adam(lr=1e-4)
    model.compile(loss=['mse', 'binary_crossentropy', 'mse'], loss_weights=[1, 1e-3, 1], optimizer=optimizer)
    
    return model

# Load the dataset first

In [16]:
import os
import cv2
import numpy as np
X = []
y = []
path = 'Data/'
for folder in os.listdir(path):
  # if folder is a directory
  if os.path.isdir(path+folder):
    for file in os.listdir(path+folder):
      img = cv2.imread(path+folder+'/'+file)
      img = cv2.resize(img,(96, 96))
      X.append(img)
      y.append(folder)

In [17]:
X = np.array(X)
y = np.array(y)

X.shape, y.shape

((414, 96, 96, 3), (414,))

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [21]:
from model.srgan import generator, discriminator
from tensorflow.keras.optimizers import Adam
import numpy as np

# Load your dataset
# ...

# Define the generator and discriminator networks
generator = generator()
discriminator = discriminator()

# Compile the discriminator model
optimizer = Adam(lr=1e-4)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

# Define the SRGAN model
srgan = build_srgan(generator, discriminator)

# Compile the SRGAN model
optimizer = Adam(lr=1e-4)
srgan.compile(loss=['mse', 'binary_crossentropy', 'binary_crossentropy'], loss_weights=[1, 1e-3, 1e-3], optimizer=optimizer)

# Train the SRGAN model
batch_size = 16
epochs = 1000

for epoch in range(epochs):
    for i in range(len(X_train) // batch_size):
        # Generate a batch of low-resolution images
        low_res_images = []
        for img in X_train[i * batch_size:(i + 1) * batch_size]:
            img_resized = cv2.resize(img, (96, 96), interpolation=cv2.INTER_LINEAR)
            low_res_images.append(img_resized)
        low_res_images = np.array(low_res_images)

        # Generate a batch of high-resolution images
        high_res_images = X_train[i * batch_size:(i + 1) * batch_size]

        # Generate a batch of fake high-resolution images
        fake_high_res_images = generator.predict(low_res_images)

        # Train the discriminator network
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))
        discriminator_loss_real = discriminator.train_on_batch(high_res_images, real_labels)
        discriminator_loss_fake = discriminator.train_on_batch(fake_high_res_images, fake_labels)
        discriminator_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)

        # Train the generator network
        perceptual_loss, content_loss, adversarial_loss = srgan.train_on_batch(low_res_images, [high_res_images, real_labels, real_labels])

        # Print the loss
        print('Epoch %d, Batch %d: Discriminator Loss = %s, Adversarial Loss = %s, Content Loss = %s, Perceptual Loss = %s' % (epoch, i, discriminator_loss, adversarial_loss, content_loss, perceptual_loss))

1/1 [==============================] - 13s 13s/step


ValueError: in user code:

    File "c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Owner\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Owner